# Elk ETL

In [8]:
#import libraries
import pandas as pd
import numpy as np
import re

#import functions
from functions import get_sample_id
from functions import panda_stripper

In [9]:
#read main table and strip
elk_table = pd.read_excel('data/Elk captures 21-22.xlsx')
elk_table = panda_stripper(elk_table)

#rename cols
elk_table.columns = ['sample_id', 'archive_id', 'collar_id', 'species', 'sex', 'capture_date', 'capture_unit', 'staging_area', 'age', 'comments']

In [10]:
elk_table.head()

,sample_id,archive_id,collar_id,species,sex,capture_date,capture_unit,staging_area,age,comments
0,BCE1132,EL21-137,51628.0,Elk,Female,2022-03-07,Book Cliffs,Bryson Canyon,23.5,NaN
1,BCE1134,EL21-138,46687.0,Elk,Female,2022-03-07,Book Cliffs,Bryson Canyon,19.5,NaN
2,BCE1135,EL21-139,46752.0,Elk,Female,2022-03-07,Book Cliffs,Bryson Canyon,10.5,NaN
3,BCE1136,EL21-140,51622.0,Elk,Female,2022-03-07,Book Cliffs,Bryson Canyon,13.5,NaN
4,BCE1138,EL21-141,46700.0,Elk,Female,2022-03-07,Book Cliffs,Bryson Canyon,22.5,NaN


## Extract and tranform lab results

### ehdv

 Epizootic Hemorrhagic Disease Virus tests come in from the PDFs in a very messy format, so we must deal with it. 


In [11]:
#read and strip table
ehdv_raw_df = pd.read_excel('data/elk_tables.xlsx', sheet_name='ehdv', usecols=[0,1])
ehdv_raw_df = panda_stripper(ehdv_raw_df)

In [12]:
ehdv_raw_df.head(15)

,test_and_result,animal
0,Test: Epizootic Hemorrhagic Disease Virus Type...,"Animal: 2, BCE1134/EL21-138"
1,Result Titer,NaN
2,Negative <20,NaN
3,Test: Epizootic Hemorrhagic Disease Virus Type...,"Animal: 2, BCE1134/EL21-138"
4,Result Titer,NaN
5,Negative <20,NaN
6,Test: Epizootic Hemorrhagic Disease Virus Type...,"Animal: 3, BCE1135/EL21-139"
7,Result Titer,NaN
8,Negative <20,NaN
9,Test: Epizootic Hemorrhagic Disease Virus Type...,"Animal: 3, BCE1135/EL21-139"


In [13]:
#select every 3rd row starting with the 1st row
ehdv_df = ehdv_raw_df.iloc[0::3]
#reset index
ehdv_df = ehdv_df.reset_index(drop=True)
#rename cols
ehdv_df.columns = ['test', 'animal']

#extract result and titer from ehdv_raw_df (i.e., every 3rd row starting with 2nd row)
df = ehdv_raw_df.iloc[2::3]
del df['animal']
#reset index
df = df.reset_index(drop=True)
df.columns = ['result']

#merge
ehdv_df = pd.concat([ehdv_df, df], axis=1)

In [17]:
ehdv_df.head()

,test,animal,result
0,Test: Epizootic Hemorrhagic Disease Virus Type...,"Animal: 2, BCE1134/EL21-138",Negative <20
1,Test: Epizootic Hemorrhagic Disease Virus Type...,"Animal: 2, BCE1134/EL21-138",Negative <20
2,Test: Epizootic Hemorrhagic Disease Virus Type...,"Animal: 3, BCE1135/EL21-139",Negative <20
3,Test: Epizootic Hemorrhagic Disease Virus Type...,"Animal: 3, BCE1135/EL21-139",Positive 40
4,Test: Epizootic Hemorrhagic Disease Virus Type...,"Animal: 3, BCE1135/EL21-139",Negative <20
